In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU


In [ ]:
file_path = '/content/Drying3.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

X = data[['Drying Time (min )', 'Velocity m/s', 'Temperatures C', 'Thickness mm', 'Sample mass in (g)']]
y = data[['MR']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model_10_layers = Sequential([
    Dense(64, input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),

    Dense(64),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),

    Dense(y_train.shape[1], activation='linear')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
reduce_lr_10 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=5e-7)
early_stop_10 = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_10_layers.compile(optimizer=Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])


In [ ]:
history = model_10_layers.fit(X_train_scaled, y_train,
                    epochs=1000,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1,
                   )

y_pred = model_10_layers.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)

model_10_layers.save('final_dnn_model.keras')



Epoch 1/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - loss: 1.7098 - mae: 1.0463 - val_loss: 0.3158 - val_mae: 0.4667
Epoch 2/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.1316 - mae: 0.8739 - val_loss: 0.2775 - val_mae: 0.4401
Epoch 3/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.9529 - mae: 0.8007 - val_loss: 0.2545 - val_mae: 0.4199
Epoch 4/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.7036 - mae: 0.6956 - val_loss: 0.2490 - val_mae: 0.4118
Epoch 5/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5783 - mae: 0.6436 - val_loss: 0.2374 - val_mae: 0.3987
Epoch 6/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5444 - mae: 0.5932 - val_loss: 0.2264 - val_mae: 0.3902
Epoch 7/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4091 - mae: 0.5060 - val_loss: 0.2157 - val_mae: 0.3803
Epoch 8/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.3485 - mae: 0.4866 - val_loss: 0.2122 - val_mae: 0.3782
Epoch 9/1000
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.199

In [6]:
import numpy as np

def predict_new(data):
    data_scaled = scaler.transform([data])
    prediction = model_10_layers.predict(data_scaled)
    return [f"{val:.4f}" for val in prediction[0]]  # تحويل القيم إلى نص وتنسيقها لأربعة أرقام عشرية

# بيانات جديدة للتنبؤ
data_example = [210, 2, 80, 7, 488.16]
prediction = predict_new(data_example)

# عرض النتائج
print("Predictions for new data:", prediction)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Predictions for new data: ['-0.0182']
